# SpatialIQ: Predicting Students' Spatial Intelligence through Machine Learning

## A Comprehensive AI-in-Education Research Study

**Author:** AI Research Developer  
**Date:** November 2025  
**Dataset Citation:** DOI: 10.21227/5qxw-bw66  
**Purpose:** Interpretable prediction and analysis of spatial intelligence in high school students using behavioral, academic, and demographic features

---

## Executive Summary

This notebook presents a rigorous machine learning pipeline designed to predict students' **Spatial Intelligence** levels (ranging from Very Low to Very High) using a rich dataset of 40 behavioral, academic, and demographic features collected from high school students. Spatial intelligence—the ability to visualize, manipulate, and reason about spatial relationships—is a critical component of cognitive development and academic achievement, particularly in STEM disciplines.

### Research Questions:
1. **What behavioral and academic patterns predict spatial intelligence?**
2. **How do demographic factors (gender, parental education, environment) influence spatial reasoning ability?**
3. **Can we build interpretable models that identify actionable insights for educators?**
4. **What ethical considerations arise from AI-driven student cognitive profiling?**

### Methodology Overview:
- **Phase 1:** Comprehensive data exploration and statistical profiling
- **Phase 2:** Intelligent feature engineering with domain knowledge
- **Phase 3:** Competitive modeling with Logistic Regression, Random Forest, XGBoost, and Neural Networks
- **Phase 4:** Deep model interpretation using SHAP explainability methods
- **Phase 5:** Actionable insights and ethical considerations

---

## Dataset Overview

The dataset comprises 398 high school students with 40 features across five categories:
- **Demographics:** Age, Gender, Class size, Environment (Urban/Suburban/Rural)
- **Socioeconomic:** Family size, Parental occupation, Parental education, Income level
- **Academic:** Major, GPA, Study time, Extra classes, Teacher assessment
- **Behavioral:** Internet usage, TV watching, Pattern recognition, Geographic familiarity
- **Gaming Preferences:** Action, Adventure, Strategy, Sport, Simulation, Role-playing, Puzzle games
- **Learning Modes:** Visual vs. Auditory, Map usage, Diagram usage, Experience with GIS

**Target Variable:** Spatial Intelligence (Categorical: VL, L, M, H, VH)


---

# Section 1: Import Required Libraries and Dataset Loading

This foundational section initializes all necessary libraries for data manipulation, visualization, machine learning, and model interpretation. We leverage industry-standard tools including pandas for data handling, scikit-learn for preprocessing and modeling, XGBoost for gradient boosting, TensorFlow for neural networks, and SHAP for model explainability. This comprehensive toolkit enables both rigorous statistical analysis and production-grade machine learning workflows.

In [ ]:
# ==================== LIBRARY IMPORTS ====================
# Core Data Science and Manipulation
import pandas as pd
import numpy as np
from pathlib import Path

# Data Preprocessing and Encoding
from sklearn.preprocessing import StandardScaler, LabelEncoder, OrdinalEncoder, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer

# Model Selection, Training, and Evaluation
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier, plot_importance
from lightgbm import LGBMClassifier

# Metrics and Evaluation
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score,
    roc_curve, auc, matthews_corrcoef
)

# Statistical Analysis
from scipy.stats import chi2_contingency, mutual_info_classif, spearmanr, pearsonr
from scipy.stats import kurtosis, skew
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Feature Importance and SHAP Interpretability
import shap
from sklearn.inspection import permutation_importance

# Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Warnings and Configuration
import warnings
warnings.filterwarnings('ignore')

# Configuration for visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✓ All libraries successfully imported")
print(f"✓ NumPy version: {np.__version__}")
print(f"✓ Pandas version: {pd.__version__}")
print(f"✓ Scikit-learn version: {pd.__version__.split('.')[0]}")
print(f"✓ SHAP library initialized for model interpretation")

## 1.1: Load and Inspect the Dataset

In this subsection, we load the Dataset.csv file and perform initial exploratory inspection to understand the data structure, dimensions, data types, and preliminary statistical characteristics. This foundational step is critical for identifying potential data quality issues, missing values, and the nature of features we'll be working with throughout the analysis.

In [ ]:
# Load the dataset from CSV file
data_path = Path("data/Dataset.csv")
df = pd.read_csv(data_path)

# Display basic information about the dataset
print("=" * 80)
print("DATASET LOADING AND INITIAL INSPECTION")
print("=" * 80)
print(f"\n✓ Dataset successfully loaded from: {data_path}")
print(f"\nDataset Shape: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Display column names and data types
print("\n" + "=" * 80)
print("COLUMN INFORMATION AND DATA TYPES")
print("=" * 80)
print("\nColumn Names and Data Types:")
print(df.info())

# Display summary statistics
print("\n" + "=" * 80)
print("FIRST 10 ROWS OF DATA")
print("=" * 80)
display(df.head(10))

print("\n" + "=" * 80)
print("SUMMARY STATISTICS")
print("=" * 80)
display(df.describe().transpose())

# Check for missing values
print("\n" + "=" * 80)
print("MISSING VALUES ANALYSIS")
print("=" * 80)
missing_counts = df.isnull().sum()
missing_percentage = (df.isnull().sum() / len(df)) * 100
missing_df = pd.DataFrame({
    'Column': missing_counts.index,
    'Missing_Count': missing_counts.values,
    'Missing_Percentage': missing_percentage.values
})
missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)

if len(missing_df) > 0:
    print(f"\nColumns with missing values ({len(missing_df)} found):")
    display(missing_df)
else:
    print("\n✓ No missing values detected in the dataset!")

# Identify target variable
print("\n" + "=" * 80)
print("TARGET VARIABLE IDENTIFICATION")
print("=" * 80)
print("\nDataset column names:")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

# The last column is typically the target (Spatial Intelligence level)
target_col = df.columns[-1]
print(f"\n✓ Identified target variable: '{target_col}'")
print(f"Unique values in target: {df[target_col].unique()}")
print(f"Target value counts:\n{df[target_col].value_counts().sort_index()}")

# Create a reference dictionary for later use
target_mapping = {val: idx for idx, val in enumerate(sorted(df[target_col].unique()))}
print(f"\nTarget ordinal mapping: {target_mapping}")

---

# Section 2: Exploratory Data Analysis and Statistical Profiling

## 2.1: Distribution Analysis of Spatial Intelligence

Spatial Intelligence is our target variable, classified into five ordinal categories ranging from Very Low (VL) to Very High (VH). Understanding the distribution of this outcome variable is crucial for identifying potential class imbalance issues and informing our choice of evaluation metrics and sampling strategies. We'll visualize this distribution through multiple perspectives: raw counts, percentages, and normalized visualizations.

In [ ]:
# Analyze the distribution of the target variable
print("=" * 80)
print("TARGET VARIABLE DISTRIBUTION ANALYSIS")
print("=" * 80)

target_distribution = df[target_col].value_counts().sort_index()
target_percentages = (target_distribution / len(df)) * 100

distribution_df = pd.DataFrame({
    'Spatial_Intelligence_Level': target_distribution.index,
    'Count': target_distribution.values,
    'Percentage': target_percentages.values,
    'Cumulative_Percentage': target_percentages.cumsum().values
})

print("\nTarget Variable Distribution:")
display(distribution_df)

# Calculate class balance metrics
class_weights = len(df) / (len(target_distribution) * target_distribution)
print(f"\nClass Balance Weights (for handling imbalance):")
for level, weight in zip(target_distribution.index, class_weights):
    print(f"  {level}: {weight:.4f}")

# Create visualization of target distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar plot with counts
axes[0].bar(target_distribution.index, target_distribution.values, color=sns.color_palette("husl", len(target_distribution)))
axes[0].set_xlabel('Spatial Intelligence Level', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Count', fontsize=12, fontweight='bold')
axes[0].set_title('Distribution of Spatial Intelligence (Counts)', fontsize=13, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)
for i, v in enumerate(target_distribution.values):
    axes[0].text(i, v + 1, str(v), ha='center', fontweight='bold')

# Pie chart with percentages
colors = sns.color_palette("husl", len(target_distribution))
wedges, texts, autotexts = axes[1].pie(
    target_distribution.values,
    labels=target_distribution.index,
    autopct='%1.1f%%',
    colors=colors,
    startangle=90
)
axes[1].set_title('Distribution of Spatial Intelligence (Percentage)', fontsize=13, fontweight='bold')
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')

plt.tight_layout()
plt.savefig('visualizations/01_target_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Target distribution visualization saved")

## 2.2: Feature Type Classification and Statistical Profiling

Before conducting deeper analysis, we need to classify features by type (numeric, categorical ordinal, categorical nominal) and understand their individual distributions. This classification will inform our encoding strategies and feature engineering decisions. We'll compute key statistical properties including skewness, kurtosis, and outlier presence for numeric features, and frequency distributions for categorical features.